In [ ]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

#import necessary libraries.
import tensorflow as tf
import numpy as np
layer = tf.keras.layers

print('check tensorflow version : ', tf.__version__)

# data load

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
print("The shape of train dataset : ", x_train.shape)
print("The shape of test dataset : ", x_test.shape)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(x_train[0], cmap='gray')
plt.show()

# 1. Sequential model

Keras에서 Sequential modeling은 신경망 구조를 만드는 가장 간단한 방법이다. 신경망의 모든 계층이 직렬 구조를 갖을 때 사용할 수 있다.
Sequential model은 높은 직관성과 가독성으로 복잡한 신경망의 하위 구조를 만들 때 활용된다.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

'''
또 다른 방식인 add 메서드를 통한 레이어 추가입니다.
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
'''

In [ ]:
# The compile step specifies the training configuration.
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

model.fit(x_train, y_train, epochs=5, validation_split=0.2)
# fit 메서드에 입력 X와 정답 Y 그리고 학습 횟수인 epochs을 결정할 수 있습니다. 
# validation_split은 학습데이터 중 일부를 자동으로 validation_set으로 배정합니다.

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
del model
tf.keras.backend.clear_session()

# tip.keras model build

keras에서 모델을 만들기 위해 알아야 할 3가지 자료형인 model, layer, tensor가 있습니다.

- model은 layer를 엮은 네트워크 객체라고 할 수 있습니다.

- 인공신경망을 구성하는 하위 계층을 의미합니다.

- tensor는 layer와 layer 사이의 입력과 출력을 의미합니다. TensorFlow는 flow graph의 형태이기 때문에 tensor는 데이터가 중간에 저장되는 일종의 다차원 배열입니다.

In [ ]:
#layer는 tf.keras.layers 아래의 클래스로 만들 수 있다.
d1 = tf.keras.layers.Dense(8, activation='relu')

In [ ]:
print(type(d1)) #레이어의 타입을 볼 수 있다.

In [ ]:
# 각 layer 클래스는 method로 init, build, call를 갖는다. 
# init은 객체가 만들어지는 단계이고, tf.keras.layers.Dense(32, activation='relu')를 선언했을 때 실행된다.
# build는 실제로 layer가 파라미터를 갖는 단계, call은 layer가 가진 parameter로 데이터를 계산하는 단계이다.

print(d1.get_weights()) #weight는 build되지 않았기 때문에 없음.

In [ ]:
# build는 layer가 첫 번째로 call됐을 때 파라미터를 생성함. 특정 데이터를 입력하기 애매하기 때문에 보통 tf.keras.Input을 사용함.
inputs = tf.keras.Input(shape=(4)) #shape를 정해주기 위한 placeholder 텐서
print('type of inputs', type(inputs))

In [ ]:
#layer의 input shape이 정해져야 layer가 build 단계에서 파라미터의 shape을 결정할 수 있음.
d1_output = d1(inputs)
print('weight shape : ', d1.get_weights()[0].shape,
      '\n-------------------------------------------------\n',
      'bias shape : ', d1.get_weights()[1].shape)

In [ ]:
d1.get_weights()[0]

In [ ]:
# build가 된 layer는 이제 입력으로 데이터를 받으면 output을 계산한다.
d1_output = d1(np.ones([1,4], dtype=np.float32))
print('d1_output의 타입', type(d1_output)) # 계산 결과의 자료형은 tensor
print('\n-------------------------------------------------\n')
print(d1_output.numpy()) #tf 2.0은 eager excution을 default로 제공하여 현재 tensor가 가지고 있는 값을 확인할 수 있음.

# 2. Functional API

Keras에서 Funtional API는 유연한 신경망을 구현할 수 있도록 한다. 신경망의 구현은 앞선 레이어의 출력텐서를 다음 레이어의 입력텐서로 선언해주는 방식으로 이루어진다. 이때 레이어 대신 모델(sequential model 포함)을 하위 구조로 사용이 가능해 계층화된 복잡한 신경망을 쉽게 구현할 수 있다.

In [ ]:
inputs = tf.keras.Input(shape=(28,28,1))  # Returns a placeholder tensor
x = tf.keras.layers.Flatten()(inputs)
# A layer instance is callable on a tensor, and returns a tensor.
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
predictions = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

In [ ]:
#make sure you have already graphviz, pydot, pydotplus libraries.
tf.keras.utils.plot_model(model, 'my_first_model_with_shape_info.png', show_shapes=True)

In [ ]:
# The compile step specifies the training configuration.
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

model.fit(x_train, y_train, epochs=5, validation_split=0.2)


In [ ]:
model.evaluate(x_test, y_test)

## model saving

In [ ]:
model.save("model.h5") #it saves your model and parameters
model.save_weights("weights.h5") #it saves your parameters only

In [ ]:
new_model = tf.kera.models.load_model("model.h5")
new_model.summary()

In [ ]:
del model
del new_model
tf.keras.backend.clear_session()

### more complex modeling

In [ ]:
#시나리오 : 영상과 텍스트(integer encoded)를 입력으로 받는 multi-modal 모델을 구현한다.
#테스크는 분류이다.

input_layer1 = tf.keras.Input(shape=(28,28,3)) #입력영상
x = tf.keras.layers.Conv2D(32, (3,3), activation='relu')(input_layer1)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(32, (3,3), activation='relu')(x)
y1 = tf.keras.layers.Flatten()(x)

input_layer2 = tf.keras.Input(shape=(50)) #time step 50의 텍스트 문장(정수 인코딩)
vocab = 2000 #단어수
x = tf.keras.layers.Embedding(vocab, 64)(input_layer2) #word embedding dimension = 64
x = tf.keras.layers.LSTM(64, activation='tanh', return_sequences=True)(x)
y2 = tf.keras.layers.LSTM(64, activation='tanh')(x)

#정보통합
y = tf.concat([y1,y2], axis=-1)

#분류기
hidden = tf.keras.layers.Dense(64, activation='relu')(y)
class_num = 5
pred = tf.keras.layers.Dense(class_num, activation='softmax')(hidden)

In [ ]:
complex_model = tf.keras.Model([input_layer1, input_layer2], pred)
tf.keras.utils.plot_model(complex_model)

# Another way?

더 유연한 모델의 기능을 구현하고 싶은 경우 Subclassing 방식으로 모델의 동작 방식을 정의해줄 수 있다.
tf.keras의 Model 클래스를 상속하여, 개발자가 추가적인 함수를 구현한다.
그러나 복잡성이 늘어나는 만큼 기존 API와의 호환성을 유의하여 사용해야 한다.


https://www.tensorflow.org/guide/keras/custom_layers_and_models


# 3. Loop training

model.fit 메서드는 편리한 인터페이스를 제공하지만 학습 알고리즘의 커스터마이징이 어렵다. 커스터마이징에는 새로운 Loss 함수 정의, 네트워크의 학습범위를 제어하는 것을 포함한다. Loop training 방식은 이름에서도 알 수 있듯이 Loop 구조 내에서 tf.GradientTape을 사용해 학습을 제어하는 방식이다.

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
def custom_loss_example(target_y, predicted_y):
  return tf.reduce_mean(tf.square(target_y - predicted_y))

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

# Split pre-trained model for customized transfer learning

In [ ]:
# You can take famous image processing architecture, resnet101.


#https://keras.io/applications/#usage-examples-for-image-classification-models
inputs = tf.keras.Input(shape=(240, 240, 3))
resnet101 = tf.keras.applications.ResNet101(include_top=False, weights='imagenet', input_tensor=inputs)

In [ ]:
#plot the architecture of resnet 101
tf.keras.utils.plot_model(resnet101, 'resnet101.png', show_shapes=True)

In [ ]:
model_input = resnet101.input

In [ ]:
model_output = resnet101.get_layer('conv2_block1_add').output

In [ ]:
new_model = tf.keras.Model(inputs=model_input, outputs=model_output)

In [ ]:
new_model.summary()